In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Absenteeism_preprocessed.csv")
data = data.drop("Unnamed: 0" ,axis=1)
data.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Months,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
#create targets for logistic regression.Logistic regresion classifies
#consider 2 classes ; moderatley absent and excessively absent
#take mean f absenteeism hours. below median is moderatley absent else excessively absent

median_absent_time = data['Absenteeism Time in Hours'].median()

targets = np.where(data['Absenteeism Time in Hours'] > median_absent_time, 1, 0) #condition, true value, false value
#merge targets to data

data["Excessive Absenteeism"] = targets
data_with_targets = data.drop(["Absenteeism Time in Hours","Distance to Work","Daily Work Load Average", "Day of Week"],axis=1) 
#removing exceive absenteesism as we have moved it to tagrets. others being dropped are because later down once examining 
#coefficients we saw that these didnt have much wieght and coeffs were close to 0
data_with_targets.head()

,Reason 1,Reason 2,Reason 3,Reason 4,Months,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


In [4]:
#select inputs
#all rows and all colums except excessive absenteesim
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [5]:
#standardize data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() #new object
#standardization is (x-mean)/std dev

#standardizing dummy variables is not a good practice. such as reason 1 2 3 4 and education
scaled_inputs = unscaled_inputs.copy()
cols_to_scale =['Months','Transportation Expense', 'Age','Body Mass Index','Children', 'Pets']
scaler.fit(unscaled_inputs[cols_to_scale])
#transform does the actual scaling
unscaled_inputs[cols_to_scale] = scaler.transform(unscaled_inputs[cols_to_scale])
scaled_inputs[cols_to_scale] = unscaled_inputs[cols_to_scale]

scaled_inputs

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


,Reason 1,Reason 2,Reason 3,Reason 4,Months,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
5,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
6,0,0,0,1,0.182726,2.092381,-1.320435,0.061825,0,-0.019280,2.843016
7,0,0,0,1,0.182726,0.568211,-0.065439,-0.878984,0,2.679969,-0.589690
8,0,0,1,0,0.182726,-1.016322,-0.379188,-0.408580,0,0.880469,-0.589690
9,0,0,0,1,0.182726,0.190942,0.091435,0.532229,1,-0.019280,0.268487


In [6]:
#split data into training and testing data to avoid overfitting
#also shuffle data to remove dependencies if any due to sequence of data

from sklearn.model_selection import train_test_split

In [7]:
#train_inputs,test_inputs, train_targets, test_targets = train_test_split(scaled_inputs,targets) #params are inputs and targets
#the above call returns 4 arrays. 1: trainign datasets with inputs. 2: training data sets with targets. 3: test data set with inputs. 4: test datasets with targets
#in the above call we did not specify division for train and test so it does 75% 25% respectively. if we want to specify ourself we do 
train_inputs,test_inputs, train_targets, test_targets = train_test_split(scaled_inputs,targets, train_size = 0.8, shuffle = True, random_state = 20) #hance the train size becomes 80% + shuffling also don within. 
#Shuffle is by default is true. # random_state if not set, we always get a different random set. if we dont want tthat we specify this so that every shuffling we get on re run is the same.
print(train_inputs.shape,train_targets.shape)

(560, 11) (560,)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
#time to model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [9]:
#train the model
logistic_regression = LogisticRegression()
logistic_regression.fit(train_inputs,train_targets) #this fits the model according to the given training data. Does all the machine learning

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [10]:
#get accuracy of your regression based on training data
logistic_regression.score(train_inputs,train_targets)

0.775

In [11]:
#check accuracy manually
#here we will get outputs and match to targets ourselves

predicted_outputs = logistic_regression.predict(train_inputs)
predicted_outputs == train_targets #if indices macth then true. mean accurate prediction for that row, else false

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [12]:
#to find accuracy % ..... True =1 and false is 0
np.sum(predicted_outputs == train_targets)/predicted_outputs.shape[0]

0.775

In [13]:
#find intercepts and coefficients
intercept = logistic_regression.intercept_[0]
coeffs = logistic_regression.coef_

In [14]:
#creating a dataframe to contain all features and their coefficients
feature_names = unscaled_inputs.columns.values
summary_table = pd.DataFrame(columns=['Feature name'], data = feature_names)
summary_table['Coefficient'] = np.transpose(coeffs)
summary_table

,Feature name,Coefficient
0,Reason 1,2.627499
1,Reason 2,0.863386
2,Reason 3,2.960507
3,Reason 4,0.663907
4,Months,0.154937
5,Transportation Expense,0.599798
6,Age,-0.172451
7,Body Mass Index,0.275685
8,Education,-0.234525
9,Children,0.342497


In [15]:
#to add intercept in the above df
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', intercept]
summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.465471
1,Reason 1,2.627499
2,Reason 2,0.863386
3,Reason 3,2.960507
4,Reason 4,0.663907
5,Months,0.154937
6,Transportation Expense,0.599798
7,Age,-0.172451
8,Body Mass Index,0.275685
9,Education,-0.234525


In [16]:
# log(odds) = b0 + b1x1 ..... b0 i intercept and b1 i coeff or weight
summary_table['Odds Ratio'] = np.exp(summary_table.Coefficient)
summary_table.sort_values('Odds Ratio', ascending = False)
# a feature with Coefficient near 0 and odds ratio near 1, this means feature is not very important. As this weight of 0 will multiply the value of feature and give 0. such features can be removed as they are uselesss
#further te coeff away from 0, the more important it is

,Feature name,Coefficient,Odds Ratio
3,Reason 3,2.960507,19.307751
1,Reason 1,2.627499,13.839121
2,Reason 2,0.863386,2.371177
4,Reason 4,0.663907,1.942367
6,Transportation Expense,0.599798,1.821751
10,Children,0.342497,1.408460
8,Body Mass Index,0.275685,1.317433
5,Months,0.154937,1.167585
7,Age,-0.172451,0.841599
9,Education,-0.234525,0.790946


In [17]:
#backward elimination. lets remove features from our model which have coeffs very close to 0 or odds very close to 1 as they have very little effect and weight
#so Daily Work Load Average, Distance to Work, Day of Week can be ignored .....  if you dont see it here it means i have already removed it and re run the code

In [18]:
#get test accuracy of your regression
logistic_regression.score(test_inputs,test_targets)
#generally test accuracy < train accuracy

0.75

In [19]:
# find the predicted probabilities of each class
# the first column shows the probability of a particular observation to be 0, while the second one - to be 1
predicted_proba = logistic_regression.predict_proba(test_inputs)

# let's check that out
predicted_proba

array([[0.71221976, 0.28778024],
       [0.58760009, 0.41239991],
       [0.44337438, 0.55662562],
       [0.77903962, 0.22096038],
       [0.08458343, 0.91541657],
       [0.33103371, 0.66896629],
       [0.29792496, 0.70207504],
       [0.12956221, 0.87043779],
       [0.78307821, 0.21692179],
       [0.74708659, 0.25291341],
       [0.49514969, 0.50485031],
       [0.22640297, 0.77359703],
       [0.07030984, 0.92969016],
       [0.73504052, 0.26495948],
       [0.30533085, 0.69466915],
       [0.55035881, 0.44964119],
       [0.55027426, 0.44972574],
       [0.53930442, 0.46069558],
       [0.40117774, 0.59882226],
       [0.05320682, 0.94679318],
       [0.69874615, 0.30125385],
       [0.77903962, 0.22096038],
       [0.41634563, 0.58365437],
       [0.41634563, 0.58365437],
       [0.2412915 , 0.7587085 ],
       [0.74317087, 0.25682913],
       [0.51065194, 0.48934806],
       [0.85703303, 0.14296697],
       [0.19934235, 0.80065765],
       [0.77903962, 0.22096038],
       [0.

In [20]:
#save the model
#everything that needs to be saved is in the logistic_regression = LogisticRegression() object
#pickle is a python module ued to convert object to character stream. in simple words we save the object to a file so that it(algo) can be used in another python notebook if needed
import pickle

with open('absenteeism_logistic_regression', 'wb') as file: #filename and wb is write byte command
    #pickling means save model in file so wrtitng is done here
    pickle.dump(logistic_regression,file) 
  

#also pickle scaler as we need to preprocess any new data entered into the model in a similar way so hence we save this for reuse in other notebook
with open('absenteeism_scaler', 'wb') as file: #filename and wb is write byte command
    pickle.dump(scaler,file) #pickling means save model in file so wrtitng is done here